In [35]:
def remove_common_prefix(grammar):
    check = True
    while check == True:
        check = False
        temp_dict = {}
        count = 0
        ch = 'A'
        for x in grammar.keys():
            productions = grammar[x]
            length = len(productions)
            arr = [False for i in range(length)]
            for i in range(0,length):
                if arr[i] == False :
                    arr[i] = True
                    temp_list = []

                    new_list = []
                    flag = False
                    for j in range(i+1,length):
                        if productions[i][0] == productions[j][0]:
                            if flag == False:
                                flag = True
                                while(chr(count+65) in grammar.keys()):
                                    count += 1
                                ch = chr(count+65)
                            arr[j] = True
                            temp_list.append(j)
                    temp_list.append(i)
                    if len(temp_list)>1:
                        for k in range(0,len(temp_list)):
                            index = temp_list[k]
                            temp = productions[index][1:]
                            new_list.append(temp)
                            productions[index] = productions[index][0] + ch
                    grammar[x] = list(set(productions))
                    if len(new_list)>0 :
                        temp_dict[ch] = new_list
        for x in temp_dict.keys():
            check = True
            grammar[x] = temp_dict[x]
    return grammar
                    


In [36]:
    def remove_left_recursion(grammar):
        hash_map = {}
        temp=0
        count = 0
        for x in grammar.keys():
            hash_map[x] = temp
            temp += 1
        temp_dict = {}
        for x in grammar.keys():
            productions = grammar[x]
            length = len(productions)
            to_be_added = []
            to_be_removed = []
            for i in range(length):
                for j in range(0,26):
                    ch = chr(65+j)
                    if ch in productions[i]:
                        if hash_map[x] > hash_map[ch]:
                            word = productions[i]
                            to_be_removed.append(productions[i])

                            temp_list = grammar[ch]
                            for k in range(len(temp_list)):
                                to_be_added.append(word.replace(ch,temp_list[k]))

            productions = [l for l in productions if l not in to_be_removed]
            productions += to_be_added
            grammar[x] = productions
            var = []
            const = []
            flag = False
            length = len(productions)
            for i in range(length):
                if productions[i][0] == x:
                    flag = True
                    var.append(productions[i][1:])
                else:
                    const.append(productions[i])

            if flag==True:
                while(chr(count+65) in grammar.keys() or chr(count+65) in temp_dict.keys()):
                    count += 1
                ch = chr(65+count)
                new_prod = []
                for l in range(len(const)):
                    new_prod.append(const[l]+ch)
                productions = new_prod
                new_prod2 = []
                for l in range(len(var)):
                    new_prod2.append(var[l]+ch)
                new_prod2.append("^")
                temp_dict[ch] = new_prod2
                grammar[x] = productions
        for x in temp_dict.keys():
            grammar[x] = temp_dict[x]

        return grammar

In [37]:

from collections import OrderedDict
fo = open("grammar.txt","r")
lines = fo.readlines()
grammar = OrderedDict()
for x in lines:
    x = x.strip("\t")
    x = x.strip("\n")
    x = x.strip(" ")
    one,two = x.split(" -> ")
    two_list = two.split("|")
    if one in grammar.keys():
        grammar[one] = grammar[one].extend(two_list)
    else:
        grammar[one] = two_list
print(grammar)
grammar = remove_left_recursion(grammar)
grammar = remove_common_prefix(grammar)


print(grammar)



OrderedDict([('E', ['E+T', 'E-T', 'T']), ('T', ['T*F', 'T/F', 'F']), ('F', ['a', '(E)'])])
OrderedDict([('E', ['TA']), ('T', ['FB']), ('F', ['a', '(TA)']), ('A', ['^', '-TA', '+TA']), ('B', ['^', '*FB', '/FB'])])


In [38]:
f = open("grammar2.txt","w")
for i in grammar.keys():
    string = i + " -> "
    temp = grammar[i]
    for j in range(0,len(temp)-1):
        string += temp[j] +" | "
    string += temp[len(temp)-1] +"\n"
    #print(string)
    f.write(string)    
f.close()

In [39]:
    import re
    import string
    import pandas as pd 



In [40]:
   def parse(input,start_symbol,parsingTable):
        flag = 0

        #appending $ to end of input
        input = input + "$"

        stack = []

        stack.append("$")
        stack.append(start_symbol)

        input_len = len(input)
        index = 0


        while len(stack) > 0:

            top = stack[len(stack)-1]

            print("Top =>",top)

            #current input
            cur_input = input[index]

            print ("cur_input => ",cur_input)

            if top == cur_input:
                stack.pop()
                index = index + 1
            else:

                #finding value for key in table
                key = top , cur_input
                print (key)

                #top of stack terminal => not accepted
                if key not in parsingTable:
                    flag = 1
                    break

                value = parsingTable[key]
                if value !='^':
                    value = value[::-1]
                    value = list(value)

                    #poping top of stack
                    stack.pop()

                    #push value chars to stack
                    for element in value:
                        stack.append(element)
                else:
                    stack.pop()

        if flag == 0:
            print ("String accepted!")
        else:
            print ("String not accepted!")



In [41]:
 def ll1(follow, productions):

        print ("\nParsing Table\n")

        table = {}
        for key in productions:
            for value in productions[key]:
                if value!='^':
                    for element in first(value, productions):
                        table[key, element] = value
                else:
                    for element in follow[key]:
                        table[key, element] = value

        for key,val in table.items():
            print (key,"=>",val)

        new_table = {}
        for pair in table:
            new_table[pair[1]] = {}

        for pair in table:
            new_table[pair[1]][pair[0]] = table[pair]


        print ("\n")
        print ("\nParsing Table in matrix form\n")
        print (pd.DataFrame(new_table).fillna('-'))
        print ("\n")

        return table

  

In [42]:
  def follow(s, productions, ans):
        if len(s)!=1 :
            return {}

        for key in productions:
            for value in productions[key]:
                f = value.find(s)
                if f!=-1:
                    if f==(len(value)-1):
                        if key!=s:
                            if key in ans:
                                temp = ans[key]
                            else:
                                ans = follow(key, productions, ans)
                                temp = ans[key]
                            ans[s] = ans[s].union(temp)
                    else:
                        first_of_next = first(value[f+1:], productions)
                        if '^' in first_of_next:
                            if key!=s:
                                if key in ans:
                                    temp = ans[key]
                                else:
                                    ans = follow(key, productions, ans)
                                    temp = ans[key]
                                ans[s] = ans[s].union(temp)
                                ans[s] = ans[s].union(first_of_next) - {'^'}
                        else:
                            ans[s] = ans[s].union(first_of_next)
        return ans

In [43]:
  def first(s, productions):
        c = s[0]
        ans = set()
        if c.isupper():
            for st in productions[c]:
                if st == '^' :
                    if len(s)!=1 :
                        ans = ans.union( first(s[1:], productions) )
                    else :
                        ans = ans.union('^')
                else :
                    f = first(st, productions)
                    ans = ans.union(x for x in f)
        else:
            ans = ans.union(c)
        return ans

In [44]:
        productions=dict()
        grammar = open("grammar2.txt", "r")
        first_dict = dict()
        follow_dict = dict()
        flag = 1
        start = ""
        for line in grammar:
            l = re.split("( |->|\n|\||)*", line)
            lhs = l[0]
            rhs = set(l[1:-1])-{''}
            if flag :
                flag = 0
                start = lhs
            productions[lhs] = rhs

        print ('\nFirst\n')
        for lhs in productions:
            first_dict[lhs] = first(lhs, productions)
        for f in first_dict:
            print (str(f) + " : " + str(first_dict[f]))
        print ("")

        print ('\nFollow\n')

        for lhs in productions:
            follow_dict[lhs] = set()

        follow_dict[start] = follow_dict[start].union('$')

        for lhs in productions:
            follow_dict = follow(lhs, productions, follow_dict)

        for lhs in productions:
            follow_dict = follow(lhs, productions, follow_dict)

        for f in follow_dict:
            print (str(f) + " : " + str(follow_dict[f]))

        ll1Table = ll1(follow_dict, productions)

        #parse("a*(a+a)",start,ll1Table)
        parse("a+a*a",start,ll1Table)

  



First

E : {'(', 'a'}
T : {'(', 'a'}
F : {'(', 'a'}
A : {'^', '-', '+'}
B : {'^', '/', '*'}


Follow

E : {'$'}
T : {'-', '$', '+', ')'}
F : {'-', '/', '*', '+', ')', '$'}
A : {'$', ')'}
B : {'-', '+', ')', '$'}

Parsing Table

('E', '(') => TA
('E', 'a') => TA
('T', '(') => FB
('T', 'a') => FB
('F', 'a') => a
('F', '(') => (TA)
('A', '$') => ^
('A', ')') => ^
('A', '-') => -TA
('A', '+') => +TA
('B', '-') => ^
('B', '+') => ^
('B', ')') => ^
('B', '$') => ^
('B', '*') => *FB
('B', '/') => /FB



Parsing Table in matrix form

   $     (  )    *    +    -    /   a
A  ^     -  ^    -  +TA  -TA    -   -
B  ^     -  ^  *FB    ^    ^  /FB   -
E  -    TA  -    -    -    -    -  TA
F  -  (TA)  -    -    -    -    -   a
T  -    FB  -    -    -    -    -  FB


Top => E
cur_input =>  a
('E', 'a')
Top => T
cur_input =>  a
('T', 'a')
Top => F
cur_input =>  a
('F', 'a')
Top => a
cur_input =>  a
Top => B
cur_input =>  +
('B', '+')
Top => A
cur_input =>  +
('A', '+')
Top => +
cur_input =>  +
Top => 

C:\Users\uditi\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
